In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [ ]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [ ]:
world_topo = data.world_110m()

## <font color="#747a7f">7.2</font> Marcas de Formas Geográficas

Para visualizar dados geográficos, o Altair oferece o tipo de marca `geoshape`. Para criar um mapa básico, nós podemos criar uma marca `geoshape` e passar nossos dados TopoJSON, os quais são então desempacotados em features GeoJSON, um para cada país do mundo:


In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplica uma cor azul padrão e usa uma projeção de mapa padrão (`equalEarth`). Nós podemos personalizar as cores e as larguras das linhas de fronteira usando propriedades padrões da marca. Usando o método `project` nós também podemos adicionar nossa própria projeção de mapa:

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados se encaixem na largura e na altura do gráfico. Nós podemos também especificar parâmetros de projeção, como `scale` (nível de zoom) e `translate` (panning), para personalizar as configurações de projeção. Aqui nós ajustamos os parâmetros `scale` e `translate` para focar na Europa:

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

*Observe como a resolução de 110m dos dados se torna aparente nesta escala. Para ver contornos da costa e fronteiras mais detalhados, nós precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e `translate` para focar o mapa em outras regiões!*



Até agora, nosso mapa mostra apenas países. Usando o operador `layer`, nós podemos combinar vários elementos de mapa. O Altair inclui *geradores de dados* que nós podemos usar para criar dados para camadas de mapa adicionais:

- O gerador de esfera (`{'sphere': True}`) oferece uma representação GeoJSON da esfera completa da terra. Nós podemos criar uma marca `geoshape` adicional que preencha a forma da Terra como uma camada de fundo.
- O gerador de gratícula (`{'graticule': ...}`) cria uma feature GeoJSON representando uma gratícula: uma grade formada por linhas de latitude e longitude. A gratícula padrão possui meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, existem meridianos a cada 90°. Essas configurações podem ser personalizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos dispor em camadas as marcas esfera, gratícula e país em uma especificação de mapa reutilizável:

In [ ]:
map = alt.layer(
    # usa a esfera da terra como a camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # adiciona uma gratícula para linhas de referência geográfica
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e então os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Nós podemos estender o mapa com uma projeção desejada e desenhar o resultado. Aqui aplicamos uma [Projeção Natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada de *esfera* oferece o fundo azul claro; a camada de *gratícula* oferece as linhas de referência geográficas brancas.

In [ ]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## <font color="#747a7f">7.3</font> Mapas de Pontos

Além dos dados *geométricos* oferecidos pelos arquivos GeoJSON ou TopoJSON, muitos datasets tabulares incluem informações geográficas na forma de campos para coordenadas de `longitude` e `latitude`, ou referências a regiões geográficas, como nomes de países, nomes de estados, códigos postais, *etc.*, os quais podem ser mapeados para coordenadas usando um [serviço de geocodificação](https://en.wikipedia.org/wiki/Geocoding). Em alguns casos, os dados de localização são ricos o suficiente para que nós possamos ver padrões significativos projetados apenas pelos pontos de dados &mdash; nenhum mapa base é necessário!

Vejamos um conjunto de dados de códigos postais de 5 dígitos nos Estados Unidos, incluindo coordenadas de `longitude` e `latitude` para cada agência dos correios, além de um campo `zip_code`.

In [ ]:
zipcodes = data.zipcodes.url
zipcodes

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/zipcodes.csv'

Nós podemos visualizar a localização de cada agência dos correios usando uma pequena marca `square` (1 pixel). No entanto, para definir as posições, nós *não* usamos canais `x` e `y`. Por quê?

Embora as projeções cartográficas mapeiem coordenadas (`longitude`, `latitude`) para coordenadas (`x`,`y`), elas podem fazê-lo de maneiras arbitrárias. Não há garantia, por exemplo, que `longitude` → `x` e `latitude` → `y`!
Ao invés disso, o Altair inclui canais de codificação especiais de `longitude` e `latitude` para lidar com coordenadas geográficas. Estes canais indicam quais campos de dados devem ser mapeados para coordenadas de `longitude` e `latitude`, e em seguida aplica uma projeção para mapear essas coordenadas para posições (`x`, `y`).

In [ ]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplica o campo chamado 'longitude' no canal longitude
    latitude='latitude:Q'    # aplica o campo chamado 'latitude' no canal latitude
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Ao plotar apenas os códigos postais, nós podemos ver o contorno dos Estados Unidos e discernir padrões significativos na densidade de agências dos correios, sem a necessidade de um mapa base ou elementos de referência adicionais!*

Nós usamos a projeção `albersUsa`, ao qual faz algumas adaptações na geometria real da Terra, com versões escalonadas do Alasca e do Havai no canto inferior esquerdo. Como não especificamos parâmetros de projeção `scale` ou `translate`, o Altair os define automaticamente para se ajustar aos dados visualizados.

Nós podemos agora seguir em frente e fazer mais perguntas sobre nosso dataset. Por exemplo, existe algum padrão na alocação dos códigos postais? Para avaliar esta questão, nós podemos adicionar uma codificação de cor com base no primeiro dígito do código postal. Primeiro, adicionamos uma transformação `calculate` para extrair o primeiro dígito e codificamos o resultado usando o canal de cor:

In [ ]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('digit:N',title='dígito')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para dar zoom em um dígito específico, adicione uma transformação de filtro para limitar os dados exibidos! Tente adicionar uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar por um único dígito e atualizar dinamicamente o mapa. E lembre-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar os valores dos dígitos!_

(Este exemplo é inspirado pela clássica visualização [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)

Nós poderíamos ainda nos perguntar o que a *sequência* dos códigos postais poderia indicar. Uma forma de explorar esta questão é conectar cada código postal consecutivo usando uma marca `line`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:

In [ ]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Nós podemos agora ver como os códigos postais se agrupam em áreas menores, indicando uma alocação hierárquica dos códigos por local, mas com uma variação notável dentro dos grupos locais.*

Se você estava prestando atenção aos nossos mapas anteriores, talvez você possa ter notado que há códigos postais sendo plotados no canto superior esquerdo! Estes correspondem a locais como Porto Rico ou Samoa Americana, os quais contêm códigos postais dos EUA, mas são mapeados para coordenadas `null` (`0`, `0`) pela projeção `albersUsa`. Além disso, Alasca e Havai podem complicar nossa visão dos segmentos de linha conectados. Em resposta, o código acima inclui um filtro adicional que remove pontos fora dos intervalos de `longitude` e `latitude` escolhidos.

*Remova o filtro acima para ver o que acontece!*

## <font color="#747a7f">7.4</font> Mapas de Símbolos

Agora, vamos combinar um mapa base e os dados plotados como camadas separadas. Vamos examinar a rede de voos comerciais dos EUA, considerando tanto os aeroportos quanto as rotas de voo. Para isso, nós precisaremos de três datasets.
Para o nosso mapa base, nós usaremos um arquivo TopoJSON dos Estados Unidos com resolução de 10m, contendo features para `states` ou `counties`:

In [ ]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

Para os aeroportos, nós usaremos um dataset com campos para as coordenadas de `longitude` e `latitude` de cada aeroporto, bem como o código do aeroporto `iata` &mdash; por exemplo, `'SEA'` para [Seattle-Tacoma International Airport](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).

In [ ]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

Por fim, nós utilizaremos um dataset de rotas de voo, que contém os campos `origin` e `destination` com os códigos IATA dos aeroportos correspondentes:

In [ ]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

Vamos começar criando um mapa base usando a projeção `albersUsa` e adicionar uma camada que traça marcas `circle` para cada aeroporto:

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_São muitos aeroportos! Obviamente, nem todos são grandes hubs._

Semelhante ao nosso dataset de códigos postais, os dados dos aeroportos incluem pontos que estão fora dos Estados Unidos continentais. Então, novamente nós vemos pontos no canto superior esquerdo. Nós poderíamos querer filtrar estes pontos, mas para isso, nós precisamos saber mais sobre eles.

*Atualize a projeção do mapa acima para `albers` &ndash; contornando o comportamento peculiar do `albersUsa` &ndash; para que as localizações reais desses pontos adicionais sejam reveladas!*

Agora, em vez de mostrar todos os aeroportos de maneira indistinta, vamos identificar os grandes hubs considerando o número total de rotas que se originam em cada aeroporto. Nós usaremos o dataset `routes` como nossa principal fonte de dados: ele contém uma lista de rotas de voo que nós podemos agregar para contar o número de rotas para cada aeroporto de `origin`.

Entretanto, o dataset `routes` não inclui as *localizações* dos aeroportos! Para complementar os dados de `routes` com as localizações, nós precisamos de uma nova transformação de dados: `lookup`. A transformação `lookup` pega o valor de um campo em um dataset principal e o usa como uma *chave* para procurar informações relacionadas em outra tabela. Neste caso, nós queremos comparar o código do aeroporto de `origin` no nosso dataset `routes` com o campo `iata` do dataset `airports` e, em seguida, extrair os campos correspondentes de `latitude` e `longitude`.

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

*Quais aeroportos dos EUA têm o maior número de rotas de saída?*

Agora que nós podemos ver os aeroportos, talvez queiramos interagir com eles para entender melhor a estrutura da rede de tráfego aéreo. Nós podemos adicionar uma camada de marca `rule` para representar os caminhos de aeroportos de `origin` para aeroportos de `destination`, o que requer duas transformações `lookup` para recuperar as coordenadas de cada ponto final. Além disso, nós podemos usar uma seleção `point` para filtrar estas rotas, de modo que apenas as rotas que se originam no aeroporto atualmente selecionado sejam exibidas.

*Partindo do mapa estático acima, você consegue construir uma versão interativa? Fique à vontade para pular o código abaixo para interagir com o mapa e pensar em como poderia construí-lo por conta própria!*


In [ ]:
# seleção interativa para o aeroporto de origem
# seleciona o aeroporto mais próximo ao cursor do mouse
origin = alt.selection_point(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# referência de dados compartilhada para transformações de pesquisa
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])

alt.layer(
    # mapa base dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas de rota do aeroporto de origem selecionado para aeroportos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtro para origem selecionada apenas
    ).transform_lookup(
        lookup='origin', from_=foreign # origem lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # destino lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # tamanho dos Aeroportos por número de rotas de saída
    # 1. agrega o dataset flights-airport
    # 2. pesquisa os dados de localização do dataset dos aeroportos
    # 3. remove Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_params(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # círculos menores no topo
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Passe o mouse sobre o mapa para explorar a rede de voos!_